<a href="https://colab.research.google.com/github/Jaime44/WorkSpace/blob/main/models/content%20filter%20colaborative/content-based-recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys

# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
    GPU = False
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive
    import tensorflow as tf
    print("Versión de TensorFlow:", tf.__version__)
    print("Dispositivos disponibles:", tf.config.list_physical_devices())

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/Models/content filter colaborative'

    path_workspace = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)

    # Verificar la GPU
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        print('GPU no encontrada')
    else:
      print(f'Encontrada GPU: {device_name}')
      GPU = True

    # Habilitar la GPU para TensorFlow
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print('Memoria de la GPU configurada dinámicamente')
    else:
        print('No se encontraron dispositivos GPU configurables')
else:
    print("El código se está ejecutando en un entorno local.")
    path_workspace ='C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/'
    path_absolute = os.getcwd().replace("\\", "/")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/Models/content filter colaborative'

datasets_path = "/datasets/"
path_absolute = path_absolute+datasets_path


sys.path.append(path_workspace)

El código se está ejecutando en Google Colab.
Versión de TensorFlow: 2.15.0
Dispositivos disponibles: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Contenido de la carpeta en Google Drive:
['datasets', 'CF_The_age_of_recommeder_System_Original.ipynb', 'Content-Based_and_Tensorflow_Recommender_System_Original.ipynb', 'CF_The_age_of_recommeder_System.ipynb', 'Content-Based_and_Tensorflow_Recommender_System.ipynb', 'content-based-recommendation_Original.ipynb', 'movie-recommendation-system.ipynb', 'content-based-recommendation.ipynb', '00_CFC_Recommender_system_movie.ipynb', '00_CFC_Recommender_system_movieCOLAB.ipynb', '00_CFC_Recommender_system_movieCOLAB_2.ipynb']
GPU no encontrada
No se encontraron dispositivos GPU configurables


## Se cargan las librerías

In [ ]:
# import Required Libraries

import pandas as pd
import numpy as np
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Se cargan los Datasets

In [ ]:
# df = pd.read_csv(path_absolute+"df_mezclado_tags_ratings_movies_links_genTags.csv")

In [ ]:
# dataFrame_onehot_encode = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs_onehot_encode.csv', sep=',')

# dataFrame_onehot_encode.head()


In [ ]:
dataFrame_without_ohe = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs.csv')
print(dataFrame_without_ohe.shape)
dataFrame_without_ohe.head()
dataFrame = dataFrame_without_ohe.copy()

<ipython-input-5-b1f63c1d4708>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dataFrame_without_ohe = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs.csv')


(25000095, 8)


In [ ]:
dataFrame_without_ohe_min_ratings = pd.read_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs_min_ratins.csv')
print(dataFrame_without_ohe_min_ratings.shape)
dataFrame_without_ohe_min_ratings.head()
dataFrame = dataFrame_without_ohe_min_ratings.copy()


(56162, 8)


In [ ]:
# dataFrame = dataFrame.sample(n=250000, random_state=42)
df = dataFrame.copy()
print(df.shape)
df.head()

(56162, 8)


,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
0,521,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1279793342,5.0
1,741,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1262230676,4.0
2,1028,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1303088217,5.0
3,2057,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1432118815,4.5
4,2730,296,"r:strong language, great cast excellent, ironi...","masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1450742665,4.0


## Ordenar el dataFrame por la columna 'timestamp' de mayor a menor, de la claifición mas reciente a la más antigua.


In [ ]:
dataFrame_sorted_by_timestamp = dataFrame.copy()

# Ordenar el DataFrame por la columna 'timestamp' de mayor a menor
dataFrame_sorted_by_timestamp = dataFrame_sorted_by_timestamp.sort_values(by='timestamp', ascending=False)

## Se eliminan las filas que tengan NaN en las columnas **tag_by_user** o **tag_genome**

In [ ]:
def contar_valores_nulos(df):
    # Cuenta los valores nulos en cada columna del DataFrame
    valores_nulos_por_columna = df.isnull().sum()
    # Cuenta los valores nulos en todo el DataFrame
    total_valores_nulos = df.isnull().sum().sum()
    # Imprime la cantidad de valores nulos por columna
    print("Valores nulos por columna:")
    print(valores_nulos_por_columna)
    # Imprime el total de valores nulos en el DataFrame
    print("\nTotal de valores nulos en el dataset:", total_valores_nulos)

def eliminar_filas_nulas(df, columna):
    # Seleccionar las filas con valores nulos en la columna deseada
    filas_con_nulos = df[df[columna].isnull()]
    # Mostrar las filas con valores nulos
    # print("Filas con valores nulos en la columna", columna, ":")
    # print(filas_con_nulos)
    # Eliminar las filas con valores nulos en la columna deseada
    df = df.dropna(subset=[columna])
    return df

In [ ]:
# contar_valores_nulos(df)
dataFrame_sorted_by_timestamp = eliminar_filas_nulas(dataFrame_sorted_by_timestamp, 'tag_genome')
# contar_valores_nulos(df)
dataFrame_sorted_by_timestamp = eliminar_filas_nulas(dataFrame_sorted_by_timestamp, 'tag_by_user')
# contar_valores_nulos(df)

In [ ]:
print(dataFrame_sorted_by_timestamp.shape)
dataFrame_sorted_by_timestamp.head()

(54671, 8)


,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
53748,73725,190401,"Scotland, coverup, netflix, dark, Netflix orig...","original, mentor, catastrophe, silly fun, grea...",Calibre (2018),Thriller,1574300676,3.0
30003,122409,1500,"hitman, rekindled love, romance, predictable, ...","comedy, hit men, hitman, off-beat comedy, assa...",Grosse Pointe Blank (1997),Comedy|Crime|Romance,1574276946,2.5
26209,160473,47,"Sloth, s.w.a.t., very good, Atmospheric, drug ...","powerful ending, police investigation, great e...",Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1574238626,4.5
19356,42965,79132,"menswear - outstanding, Orriginal screenplay, ...","complex, dreams, complicated, visually appeali...",Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,1574238008,5.0
38586,96399,2527,"AI, wissenschaftliche sci-fi, cyborgs, chase, ...","futuristic, robot, androids, future, robots",Westworld (1973),Action|Sci-Fi|Thriller|Western,1574219255,2.5


## Se toma un subDataset del original

In [ ]:
dataFrame = dataFrame_sorted_by_timestamp.copy()
#Tomar una submuestra
n_samples = round(dataFrame.shape[0] * 0.30)
dataFrame = dataFrame.sample(n=n_samples, random_state=42)

In [ ]:
# content_df = df[['title', 'genres', 'tag_by_user', 'tag_genome','rating']]
content_df = dataFrame[['userId', 'movieId','title', 'genres', 'tag_by_user', 'tag_genome', 'timestamp','rating']]

In [ ]:
content_df.head(4)

,userId,movieId,title,genres,tag_by_user,tag_genome,timestamp,rating
45916,18330,6975,Funny Games (1997),Drama|Horror|Thriller,"vacation, child in peril, Pretentious, dark hu...","violence, brutality, visceral, violent, distur...",1559351518,4.5
45660,43736,100383,Side Effects (2013),Crime|Drama|Mystery|Thriller,"corruption, slow start, drugs, prescription me...","psychological, conspiracy, psychology, twists ...",1368293907,3.5
49660,53413,192283,Crazy Rich Asians (2018),Comedy,"romcom, romantic comedy, romance, Happy, singa...","culture clash, comedy, based on a book, visual...",1561835637,4.0
10247,142418,1732,"Big Lebowski, The (1998)",Comedy|Crime,"drugs, Best Movie Ever, girls, Nudity (Full Fr...","quotable, cult classic, imdb top 250, off-beat...",1374245758,3.0


In [ ]:
print(content_df.shape)

(16401, 8)


## Funciones de limpieza de texto en Python

In [ ]:
import re
import string

def remove_bars(text):
    """
    Elimina las barras verticales '|' del texto y las reemplaza por espacios.
    Parámetros:
    text (str): El texto de entrada que contiene barras verticales que se eliminarán.
    Retorna:
    str: El texto con las barras verticales eliminadas.
    """
    cleaned = re.sub(r'\|', ' ', text)
    return cleaned

def separate(text):
    """
    Divide el texto por comas, limpia cada parte eliminando el texto dentro de paréntesis, dígitos, espacios en blanco, puntuación, y lo convierte a minúsculas.
    Parámetros:
    text (str): El texto de entrada que contiene partes separadas por comas.
    Retorna:
    str: El texto limpiado y concatenado con cada parte separada por un espacio.
    """
    clean_text = []
    for t in text.split(','):
        cleaned = re.sub('\(.*\)', '', t) # Elimina el texto dentro de paréntesis
        cleaned = cleaned.translate(str.maketrans('','', string.digits))
        cleaned = cleaned.replace(' ', '')
        cleaned = cleaned.translate(str.maketrans('','', string.punctuation)).lower()
        clean_text.append(cleaned)
    return ' '.join(clean_text)

def remove_punc(text):
    """
    Elimina la puntuación y los dígitos del texto y lo convierte a minúsculas.
    Parámetros:
    text (str): El texto de entrada que se limpiará.
    Retorna:
    str: El texto limpiado con la puntuación y los dígitos eliminados, convertido a minúsculas.
    """
    try:
        cleaned = text.translate(str.maketrans('','', string.punctuation)).lower()
        clean_text = cleaned.translate(str.maketrans('','', string.digits))
    except Exception as e:
        print(f"ERROR -----------------------------------------> {e} AND {text}")
    return clean_text


In [ ]:
# Ejemplo de uso: Pixar Animation Studios Warner Bros., Lancaster Gate
# text_with_bars = "oldie but goodie"
# text_with_bars = "Pixar Animation Studios"
text_with_bars = "Warner Bros., Lancaster Gate""""  """
cleaned_text = remove_punc(text_with_bars)
print(cleaned_text)

warner bros lancaster gate  


## Se eliminan signos de puntuacin de genres, tag_by_user, tag_genome y barras de genres

In [ ]:
content_df_aux = content_df.copy()
content_df_aux['genres'] = (content_df_aux['genres'].apply(remove_bars)).apply(remove_punc)
content_df_aux['tag_by_user'] = content_df_aux['tag_by_user'].apply(remove_punc)
content_df_aux['tag_genome'] = content_df_aux['tag_genome'].apply(remove_punc)
# content_df_aux.set_index('title', inplace=True)
content_df_aux = content_df_aux.reset_index()

In [ ]:
content_df_aux.shape

(16401, 9)

In [ ]:
content_df = content_df_aux.copy()
content_df.head()

,index,userId,movieId,title,genres,tag_by_user,tag_genome,timestamp,rating
0,45916,18330,6975,Funny Games (1997),drama horror thriller,vacation child in peril pretentious dark humor...,violence brutality visceral violent disturbing,1559351518,4.5
1,45660,43736,100383,Side Effects (2013),crime drama mystery thriller,corruption slow start drugs prescription medic...,psychological conspiracy psychology twists tu...,1368293907,3.5
2,49660,53413,192283,Crazy Rich Asians (2018),comedy,romcom romantic comedy romance happy singapore...,culture clash comedy based on a book visually ...,1561835637,4.0
3,10247,142418,1732,"Big Lebowski, The (1998)",comedy crime,drugs best movie ever girls nudity full fronta...,quotable cult classic imdb top offbeat comedy...,1374245758,3.0
4,46394,44616,140267,The Witch (2015),horror,annoying accents prospect america folk horror ...,witch psychological dark creepy horror,1460265987,3.5


## Se crea la bag of words con las columnas genres, tag_by_user y tag_genome

In [ ]:
df_aux = content_df.copy()
df_aux = df_aux[['genres', 'tag_by_user', 'tag_genome']]
df_aux['bag_of_words'] = ''
df_aux.loc[:, 'bag_of_words'] = df_aux.loc[:, df_aux.columns[0:]].apply(lambda x: ' '.join(x), axis=1)
# content_df.set_index('original_title', inplace=True)
columnas_a_eliminar = ['tag_by_user', 'tag_genome']
# Elimina las columnas especificadas del DataFrame 'data'
content_df = content_df.drop(columnas_a_eliminar, axis=1)

content_df['genres'] = df_aux['bag_of_words']
content_df.rename(columns={'genres': 'bag_of_words'}, inplace=True)

content_df.head()

,index,userId,movieId,title,bag_of_words,timestamp,rating
0,45916,18330,6975,Funny Games (1997),drama horror thriller vacation child in peril ...,1559351518,4.5
1,45660,43736,100383,Side Effects (2013),crime drama mystery thriller corruption slow s...,1368293907,3.5
2,49660,53413,192283,Crazy Rich Asians (2018),comedy romcom romantic comedy romance happy si...,1561835637,4.0
3,10247,142418,1732,"Big Lebowski, The (1998)",comedy crime drugs best movie ever girls nudit...,1374245758,3.0
4,46394,44616,140267,The Witch (2015),horror annoying accents prospect america folk ...,1460265987,3.5


## Creando la Matriz TF-IDF Matrix

La matriz TF-IDF es una herramienta esencial en sistemas de recomendación basados en contenido, ya que ayuda a calcular la similitud entre elementos de contenido utilizando su contenido textual, lo que mejora la precisión de las recomendaciones al encontrar elementos que comparten características semánticas.

In [ ]:
df = content_df.copy()

In [ ]:
df.shape

(16401, 7)

In [ ]:
df["bag_of_words"].head()

0    drama horror thriller vacation child in peril ...
1    crime drama mystery thriller corruption slow s...
2    comedy romcom romantic comedy romance happy si...
3    comedy crime drugs best movie ever girls nudit...
4    horror annoying accents prospect america folk ...
Name: bag_of_words, dtype: object

In [ ]:
df["bag_of_words"].isnull().sum()

0

In [ ]:
# Rellenemos el valor nulo de la variable cin bag_of_words con nada para evitar errores en los siguientes pasos
df['bag_of_words'] = df['bag_of_words'].fillna('')

In [ ]:
df["bag_of_words"].isnull().sum()

0

### DIVIDO EL CONJUNTO DE DATOS EN ENTRENAMIENTO Y OTRO DE PRUEBA

In [ ]:
# Convertir la colección de textos (en este caso, la bolsa de palabras)
# en una matriz TF-IDF.

# Vamos a eliminar las construcciones como a, an, the, and,
# pero que no tienen sentido para nosotros de nuestros DataFrames.
tfidf = TfidfVectorizer(stop_words="english")

In [ ]:
# ajustar y transformar según el objeto tfidf
# Los de las filas son textos 'bolsa de palabras'. Los de las columnas son palabras únicas.

# Crear la matriz TF-IDF (Term Frequency-Inverse Document Frequency) a partir de la columna
# 'bag_of_words' del conjunto de datos. La matriz TF-IDF es una representación numérica
# de los documentos en función de la frecuencia de las palabras que contienen y de
# su importancia en el conjunto de documentos.
tfidf_matrix = tfidf.fit_transform(df['bag_of_words'])

In [ ]:
# Cada fila representa un documento (en este caso, una película) y
# cada columna representa una palabra única en el conjunto de datos.

# Los valores en la matriz son las puntuaciones TF-IDF para cada palabra en cada documento.
tfidf_matrix.shape

(16401, 24331)

In [ ]:
#Si queremos ver todas las palabras únicas de las columnas

# Filas: Cada fila de la matriz corresponde a una película en el conjunto de datos.
# Por ejemplo, si tienes 10,000 películas, habrá 10,000 filas en la matriz.

# Columnas: Cada columna representa una palabra única en el conjunto de datos.
# La cantidad de columnas es igual al número total de palabras únicas en todas
# las bolsas de palabras de películas.

# Valores: Los valores en la matriz son las puntuaciones TF-IDF para cada palabra en cada película.
# Estas puntuaciones miden la importancia relativa de una palabra en un documento en comparación
# con su frecuencia en el conjunto de documentos. Valores más altos indican que la palabra
# es más importante en el contexto de ese documento.

tfidf.get_feature_names_out()


array(['aa', 'aaliyah', 'aamir', ..., '扭曲', '淘金记', '莫声版'], dtype=object)

In [ ]:
# tfidf scores
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Creación de la matriz de similitud del coseno

In [ ]:
# calcula la similitud coseno entre todas las filas de la matriz TF-IDF.
# La similitud coseno mide el coseno del ángulo entre dos vectores y
# proporciona una medida de cuán similares son dos documentos.

# Calcula cos sim para todos los pares de documentos posibles uno por uno. En la matriz cosine_sim,
# cada película tiene similitudes entre sí

if IN_COLAB and GPU:
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)

    import cupy as cp
    # Convertir la matriz TF-IDF a cupy array
    tfidf_matrix_cupy = cp.array(tfidf_matrix.toarray())
    # Calcular la similitud del coseno utilizando cupy.matmul
    cosine_sim_cupy = cp.matmul(tfidf_matrix_cupy, tfidf_matrix_cupy.T)
    cosine_sim = cosine_sim_cupy
else:
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

### OTRA FORMA DE HACER LA SMILITUD DEL COSENO

Calcula el producto punto (producto interno) entre pares de vectores o matrices de vectores.
En el contexto de matrices, como la matriz TF-IDF, linear_kernel calcula el producto punto entre cada par de filas (documentos) en la matriz.

In [ ]:
# from sklearn.metrics.pairwise import linear_kernel
# linear_similarity_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)
# linear_similarity_matrix.shape


In [ ]:
# cosine_sim[i, j] representa la similitud coseno entre la película con el índice i y
# la película con el índice j en el conjunto de datos.

cosine_sim.shape

(16401, 16401)

In [ ]:
cosine_sim

array([[1.        , 0.00564405, 0.01303442, ..., 0.0055089 , 0.02006233,
        0.01918406],
       [0.00564405, 1.        , 0.00178691, ..., 0.01697333, 0.00824645,
        0.03850426],
       [0.01303442, 0.00178691, 1.        , ..., 0.01802644, 0.00764575,
        0.00745447],
       ...,
       [0.0055089 , 0.01697333, 0.01802644, ..., 1.        , 0.01236511,
        0.01999701],
       [0.02006233, 0.00824645, 0.00764575, ..., 0.01236511, 1.        ,
        0.0577744 ],
       [0.01918406, 0.03850426, 0.00745447, ..., 0.01999701, 0.0577744 ,
        1.        ]])

In [ ]:
cosine_sim[1].shape

(16401,)

# Recomendaciones basada en la similitud del coseno

In [ ]:
# Vamos a crear una serie pd de índices y nombres de películas
# se crea una Serie de Pandas llamada indices donde los índices son
# los títulos de las películas y los valores son los índices originales del DataFrame df.
indices = pd.Series(df.index, index=df['title'])

In [ ]:
indices.head()

title
Funny Games (1997)          0
Side Effects (2013)         1
Crazy Rich Asians (2018)    2
Big Lebowski, The (1998)    3
The Witch (2015)            4
dtype: int64

In [ ]:
indices.shape

(16401,)

In [ ]:
# contemos la información del índice de las películas y simplifiquemos las películas más repetitivas a las más recientes
indices.index.value_counts().head()

Star Wars: Episode IV - A New Hope (1977)    207
Pulp Fiction (1994)                          107
Forrest Gump (1994)                           85
Shawshank Redemption, The (1994)              85
Matrix, The (1999)                            83
Name: title, dtype: int64

### Se eliminan las películas duplicadas del índice

In [ ]:
# Se eliminan las películas duplicadas del índice, conservando solo la última aparición de cada película.
# Esto se hace para evitar ambigüedades y asegurar que cada película tenga un índice único.
indices = indices[~indices.index.duplicated(keep='last')]

In [ ]:
indices.index.value_counts().head()

Night of the Comet (1984)     1
Border (2018)                 1
Sinister (2012)               1
Hacksaw Ridge (2016)          1
Million Dollar Baby (2004)    1
Name: title, dtype: int64

In [ ]:
indices.shape

(4186,)

In [ ]:
indices

title
Night of the Comet (1984)                                   5
Splinter (2008)                                            10
Dinner Game, The (Dîner de cons, Le) (1998)                16
Decalogue, The (Dekalog) (1989)                            26
Specials, The (2000)                                       37
                                                        ...  
About Time (2013)                                       16396
Broken Arrow (1996)                                     16397
I, Robot (2004)                                         16398
Spirited Away (Sen to Chihiro no kamikakushi) (2001)    16399
Lives of Others, The (Das leben der Anderen) (2006)     16400
Length: 4186, dtype: int64

### Pelicula a recomendar

In [ ]:
movie_title = "Wolf of Wall Street, The (2013)"

In [ ]:
# Asigno el índice de la película "Wolf of Wall Street, The (2013)" a la variable
movie_index = indices[movie_title]
movie_index

15645

### Para ver en qué se parece la película anterior a las demás películas

In [ ]:
cosine_sim[movie_index]

array([0.07075599, 0.05590135, 0.04543712, ..., 0.03386536, 0.03700274,
       0.07106033])

In [ ]:
cosine_sim[movie_index].shape

(16401,)

# Función de recomendación basada en contenido, dada una pelicula.

In [ ]:
def content_based_recommender(title, cosine_sim, dataframe, num_recomendations=10):
    """
    Genera recomendaciones basadas en contenido para una película específica.
    Parameters:
    title (str): El título de la película para la cual se generarán las recomendaciones.
    cosine_sim (array-like): Matriz de similitud coseno entre películas.
    dataframe (DataFrame): DataFrame que contiene la información de las películas.
    num_recomendations (int): El número de recomendaciones a generar (por defecto es 10).
    Returns:
    Series: Una Serie de Pandas que contiene los títulos de las películas recomendadas.
    """
    # Crear índices basados en los títulos de las películas
    indices = pd.Series(dataframe.index, index=dataframe['title'])
    indices = indices[~indices.index.duplicated(keep='last')]  # Eliminar índices duplicados

    # Capturar el índice de la película seleccionada
    movie_index = indices[title]


    # Calcular los puntajes de similitud basados en el título
    if IN_COLAB and GPU:
        similarity_scores = pd.DataFrame(cosine_sim[movie_index].get(), columns=["score"])
    else:
        similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])


    # Ordenar los índices de las películas por puntaje de similitud
    movie_indices_sort = similarity_scores.sort_values("score", ascending=False).index


    # Eliminar títulos duplicados y excluir la película seleccionada
    duplicated_titles = dataframe.loc[movie_indices_sort, 'title'].duplicated(keep='first')
    indices_sin_duplicados = movie_indices_sort[~duplicated_titles]
    filas_sin_duplicados_y_exclusion = dataframe.loc[indices_sin_duplicados]
    filas_sin_duplicados_y_exclusion = filas_sin_duplicados_y_exclusion[filas_sin_duplicados_y_exclusion['title'] != title]

    # Devolver los títulos de las películas recomendadas
    return filas_sin_duplicados_y_exclusion['title'].head(num_recomendations)


In [ ]:
# content_based_recommender("Wolf of Wall Street, The (2013)", cosine_sim, df)

In [ ]:
# content_based_recommender("Matrix, The (1999)", cosine_sim, df)

In [ ]:
# content_based_recommender("Star Wars: Episode IV - A New Hope (1977)", cosine_sim, df)

In [ ]:
# content_based_recommender('Pulp Fiction (1994)', cosine_sim, df)

In [ ]:
def get_user_preferred_movies(user_id, dataF, threshold=3.5):
    """
    Obtiene las películas preferidas por un usuario, basado en un umbral de calificación,
    y las ordena por calificación de manera descendente para priorizar las más relevantes.

    Args:
    user_id (int): ID del usuario.
    dataF (DataFrame): DataFrame que contiene las calificaciones de las películas.
    threshold (float): Umbral de calificación para considerar una película como preferida.

    Returns:
    list: Lista de IDs de películas preferidas por el usuario, ordenadas por calificación descendente.
    """
    # Filtra el DataFrame por el ID del usuario y el umbral de calificación,
    # y luego ordena los resultados por calificación de manera descendente.
    filtered_df = dataF[(dataF['userId'] == user_id) & (dataF['rating'] >= threshold)].sort_values(by='rating', ascending=False)

    # Extrae los IDs de las películas del DataFrame filtrado y ordenado.
    preferred_movies = filtered_df['movieId'].tolist()

    return preferred_movies


In [ ]:
def get_recommendations_for_user(dataF, preferred_movies, indices, cosine_sim, k=10):
    """
    Genera recomendaciones para un usuario basado en sus películas preferidas.

    Args:
    df (DataFrame): DataFrame que contiene las películas y sus características.
    preferred_movies (plist): Lista con los indices de peliculas rlevantes por el usuario.
    cosine_sim (ndarray): Matriz de similitud del coseno entre las películas.
    k (int): Número de recomendaciones a generar.

    Returns:
    list: Lista de títulos de películas recomendadas.
    """

    # Asegura que no divides por cero
    if len(preferred_movies) == 0:
        return []  # Retorna una lista vacía si no hay películas preferidas

    max_recs_per_preferred_movie = max(1, k // len(preferred_movies))

    # Inicializa un DataFrame para todas las recomendaciones.
    recommendations = []

    for movie_id in preferred_movies:
        movie_title = dataF[dataF['movieId'] == movie_id]['title'].iloc[0]
        if movie_title in indices:
            idx = indices[movie_title]
            sim_scores = sorted(list(enumerate(cosine_sim[idx])), key=lambda x: x[1], reverse=True)
            filtered_sim_scores = [(dataF.loc[item[0], 'movieId'], item[1]) for item in sim_scores if dataF.loc[item[0], 'title'] != movie_title][:max_recs_per_preferred_movie]

            recommendations.extend(filtered_sim_scores)

    # Convertir a DataFrame y agrupar por 'movieId' para eliminar duplicados antes de unir con los títulos.
    rec_df = pd.DataFrame(recommendations, columns=['movieId', 'score']).drop_duplicates(subset=['movieId'])
    rec_df = rec_df.merge(df[['movieId', 'title']].drop_duplicates(), on='movieId')

    # Calcular el promedio de las puntuaciones de similitud si hay múltiples entradas para una película.
    rec_df = rec_df.groupby('title', as_index=False)['score'].mean()

    # Asegura que la columna 'score' es de tipo numérico
    rec_df['score'] = pd.to_numeric(rec_df['score'], errors='coerce')

    # Seleccionar las top-k recomendaciones.
    top_recommendations = rec_df.nlargest(k, 'score')['title'].tolist()

    return top_recommendations


In [ ]:
user_id_random = random.choice(df['userId'])
# df[(df['userId'] == user_id_random)]

In [ ]:
indices = pd.Series(df.index, index=df['title'])
indices = indices[~indices.index.duplicated(keep='last')]  # Eliminar índices duplicados

In [ ]:
# Crear índices basados en los títulos de las películas
k=10

preferred_movies = get_user_preferred_movies(user_id_random, df)
recommendations = get_recommendations_for_user(df, preferred_movies,indices, cosine_sim, k)

recommendations

['High School Musical 3: Senior Year (2008)',
 'Late Autumn (Akibiyori) (1960)',
 'DiG! (2004)',
 'Legally Blonde 2: Red, White & Blonde (2003)',
 'Hot Shots! (1991)',
 'Mirror, The (Zerkalo) (1975)',
 'Bringing Up Baby (1938)',
 'Lost Highway (1997)',
 'Must Love Dogs (2005)',
 'Zama (2017)']

In [ ]:
def calculate_metrics_at_k(preferred_movies, recommended_movies, k):
    # Filtra las recomendaciones a las top-k
    recommended_at_k = recommended_movies[:k]

    # Verdaderos positivos: ítems recomendados que el usuario prefiere
    TP = len(set(preferred_movies) & set(recommended_at_k))

    # Falso positivo: ítems recomendados que el usuario no prefiere
    FP = len(set(recommended_at_k) - set(preferred_movies))

    # Falso negativo: ítems preferidos no recomendados
    FN = len(set(preferred_movies) - set(recommended_at_k))

    # Precision@k
    precision_at_k = TP / (TP + FP) if TP + FP != 0 else 0

    # Recall@k
    recall_at_k = TP / (TP + FN) if TP + FN != 0 else 0

    # F1@k
    if precision_at_k + recall_at_k != 0:
        f1_at_k = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k)
    else:
        f1_at_k = 0

    return precision_at_k, recall_at_k, f1_at_k


In [ ]:
k = 10

preferred_movies = get_user_preferred_movies(user_id_random, df)
recommended_movies = get_recommendations_for_user(df, preferred_movies,indices, cosine_sim, k)

precision_at_k,recall_at_k, f1_score =calculate_metrics_at_k(preferred_movies, recommended_movies, 3)

print(f"Precisión@K: {precision_at_k}")
print(f"Recall@K: {recall_at_k}")
print(f"F1 Score: {f1_score}")

Precisión@K: 0.0
Recall@K: 0.0
F1 Score: 0


In [ ]:
def calculate_global_metrics(dataF, cosine_sim, k=10, threshold=3.5):
    """
    Calcula las métricas globales para todo el conjunto de datos.

    Args:
    df (DataFrame): DataFrame que contiene las películas y sus calificaciones.
    cosine_sim (ndarray): Matriz de similitud del coseno entre las películas.
    k (int): Número de recomendaciones a generar.
    threshold (float): Umbral de calificación para considerar una película como preferida.

    Returns:
    dict: Diccionario con las métricas promedio globales.
    """
    users = dataF['userId'].unique()
    total_precision = 0
    total_recall = 0
    total_f1 = 0

    indices = pd.Series(dataF.index, index=dataF['title'])
    indices = indices[~indices.index.duplicated(keep='last')]  # Eliminar índices duplicados

    for user_id in users:
        preferred_movies = get_user_preferred_movies(user_id, dataF, threshold)
        recommended_titles = get_recommendations_for_user(dataF, preferred_movies, indices, cosine_sim, k)

        # Transforma los títulos recomendados en IDs de películas para calcular las métricas
        recommended_movies = dataF[dataF['title'].isin(recommended_titles)]['movieId'].unique().tolist()

        precision_at_k, recall_at_k, f1_at_k = calculate_metrics_at_k(preferred_movies, recommended_movies, k//2)

        total_precision += precision_at_k
        total_recall += recall_at_k
        total_f1 += f1_at_k

    # Calcula las métricas promedio
    avg_precision = total_precision / len(users)
    avg_recall = total_recall / len(users)
    avg_f1 = total_f1 / len(users)

    return {
        'average_precision@k': avg_precision,
        'average_recall@k': avg_recall,
        'average_f1@k': avg_f1
    }

In [ ]:
# Suponiendo que `df` es tu DataFrame y `cosine_sim` es tu matriz de similitud
# y que `k` y `threshold` están definidos como desees
global_metrics = calculate_global_metrics(df, cosine_sim, k=10, threshold=3.5)
print(global_metrics)

{'average_precision@k': 0.01733612273361229, 'average_recall@k': 0.012409043100203309, 'average_f1@k': 0.012759503266768635}
